In [20]:
#単語分割

import MeCab

tagger = MeCab.Tagger(" /var/lib/mecab/dic/mecab-ipadic-neologd")
# 形態素解析 
result = tagger.parse("明日晴れるかな。")
print(result)

明日	名詞,副詞可能,*,*,*,*,明日,アシタ,アシタ
晴れる	動詞,自立,*,*,一段,基本形,晴れる,ハレル,ハレル
か	助詞,副助詞／並立助詞／終助詞,*,*,*,*,か,カ,カ
な	助詞,終助詞,*,*,*,*,な,ナ,ナ
。	記号,句点,*,*,*,*,。,。,。
EOS



In [3]:
#ストップワード

import MeCab
tagger = MeCab.Tagger("-Ochasen")
tagger.parse("") 
# 形態素解析結果をリストで取得
node = tagger.parseToNode("明日晴れるかな。")

result = []
while node is not None:
    # 品詞情報取得
    hinshi = node.feature.split(",")[0]
    if  hinshi in ["名詞"]:
        # 表層形の取得
        result.append(node.surface)
    elif hinshi in ["動詞", "形容詞"]:
        # 形態素情報から原形情報を取得
        result.append(node.feature.split(",")[6])
    node = node.next
    
print(result)

['明日', '晴れる']


In [10]:
#作品分類用のデータを[aozora.model]として作成

import zipfile
import os.path
import urllib.request as req
import MeCab
from gensim import models
from gensim.models.doc2vec import TaggedDocument

#Mecabの初期化
mecab = MeCab.Tagger()
mecab.parse("")

#学習対象とする青空文庫の作品リスト
list = [
    {"auther":{
        "name":"宮澤 賢治",
        "url":"https://www.aozora.gr.jp/cards/000081/files/"}, 
     "books":[
        {"name":"銀河鉄道の夜","zipname":"43737_ruby_19028.zip"},
        {"name":"注文の多い料理店","zipname":"1927_ruby_17835.zip"},
        {"name":"セロ弾きのゴーシュ","zipname":"470_ruby_3987.zip"},
        {"name":"やまなし","zipname":"46605_ruby_29758.zip"},
        {"name":"どんぐりと山猫","zipname":"43752_ruby_17595.zip"},
    ]},
    {"auther":{
        "name":"芥川 竜之介",
        "url":"https://www.aozora.gr.jp/cards/000879/files/"}, 
     "books":[
        {"name":"羅生門","zipname":"127_ruby_150.zip"},
        {"name":"鼻","zipname":"42_ruby_154.zip"},
        {"name":"河童","zipname":"69_ruby_1321.zip"},
        {"name":"歯車","zipname":"42377_ruby_34744.zip"},
        {"name":"老年","zipname":"131_ruby_241.zip"},
    ]},
    {"auther":{
        "name":"太宰 治",
        "url":"https://www.aozora.gr.jp/cards/000035/files/"}, 
     "books":[
        {"name":"斜陽","zipname":"1565_ruby_8220.zip"},
        {"name":"走れメロス","zipname":"1567_ruby_4948.zip"},
        {"name":"津軽","zipname":"2282_ruby_1996.zip"},
        {"name":"お伽草紙","zipname":"307_ruby_3042.zip"},
        {"name":"人間失格","zipname":"301_ruby_5915.zip"},
    ]},
    {"auther":{
        "name":"夏目 漱石",
        "url":"https://www.aozora.gr.jp/cards/000148/files/"}, 
     "books":[
        {"name":"吾輩は猫である","zipname":"789_ruby_5639.zip"},
        {"name":"坊っちゃん","zipname":"752_ruby_2438.zip"},
        {"name":"草枕","zipname":"776_ruby_6020.zip"},
        {"name":"虞美人草","zipname":"761_ruby_1861.zip"},
        {"name":"三四郎","zipname":"794_ruby_4237.zip"},
    ]},
]

#作品リストを取得してループ処理に渡す
def book_list():
    for novelist in list:
        auther = novelist["auther"]
        for book in novelist["books"]:
            yield auther, book
        
#Zipファイルを開き、中の文書を取得する
def read_book(auther, book):
    zipname = book["zipname"]
    #Zipファイルが無ければ取得する
    if not os.path.exists(zipname):
        req.urlretrieve(auther["url"] + zipname, zipname)
    zipname = book["zipname"]
    #Zipファイルを開く
    with zipfile.ZipFile(zipname,"r") as zf:
        #Zipファイルに含まれるファイルを開く。今回のZIPは一つのテキストファイルのみ含
        for filename in zf.namelist():
            with zf.open(filename,"r") as f:
                #今回読むファイルはShift-JISなので指定してデコードする
                return f.read().decode("shift-jis")

#引数のテキストを分かち書きして配列にする 
def split_words(text):
    node = mecab.parseToNode(text)
    wakati_words = []
    while node is not None:
        hinshi = node.feature.split(",")[0]
        if  hinshi in ["名詞"]:
            wakati_words.append(node.surface)
        elif hinshi in ["動詞", "形容詞"]:
            wakati_words.append(node.feature.split(",")[6])
        node = node.next
    return wakati_words

#作品リストをDoc2Vecが読めるTaggedDocument形式にし、配列に追加する
documents = []
#作品リストをループで回す
for auther, book in book_list():
    #作品の文字列を取得
    words = read_book(auther, book)
    #作品の文字列を分かち書きに
    wakati_words = split_words(words)
    #TaggedDocumentの作成　文書=分かち書きにした作品　タグ=作者:作品名
    document = TaggedDocument(
        wakati_words, [auther["name"] + ":" + book["name"]])
    documents.append(document)
    
#TaggedDocumentの配列を使ってDoc2Vecの学習モデルを作成
model = models.Doc2Vec(
    documents, dm=1, vector_size=300, window=5, min_count=1)

#Doc2Vecの学習モデルを保存
model.save('aozora.model')

print("モデル作成完了")

モデル作成完了


In [4]:
#「aozora.model」の分類

import urllib.request as req
import zipfile
import os.path 
import MeCab
from gensim import models

#Mecabの初期化
mecab = MeCab.Tagger()
mecab.parse("")

#保存したDoc2Vec学習モデルを読み込み 
model = models.Doc2Vec.load('aozora.model')

#分類用のZipファイルを開き、中の文書を取得する 
def read_book(url, zipname):
    if not os.path.exists(zipname):
        req.urlretrieve(url, zipname)

    with zipfile.ZipFile(zipname,"r") as zf:
        for filename in zf.namelist():
            with zf.open(filename,"r") as f:
                return f.read().decode("shift-jis")

#引数のテキストを分かち書きして配列にする
def split_words(text):
    node = mecab.parseToNode(text)
    wakati_words = []
    while node is not None:
        hinshi = node.feature.split(",")[0]
        if  hinshi in ["名詞"]:
            wakati_words.append(node.surface)
        elif hinshi in ["動詞", "形容詞"]:
            wakati_words.append(node.feature.split(",")[6])
        node = node.next
    return wakati_words

#引数のタイトル、URLの作品を分類する 
def similar(title, url):
    zipname = url.split("/")[-1]
        
    words = read_book(url, zipname)
    wakati_words = split_words(words)
    vector = model.infer_vector(wakati_words)
    print("--- 「" + title + '」と似た作品は? ---')
    print(model.docvecs.most_similar([vector],topn=3))
    print("")

#各作家の作品を１つずつ分類 
similar("宮沢 賢治:よだかの星",
        "https://www.aozora.gr.jp/cards/000081/files/473_ruby_467.zip")

similar("芥川 龍之介:犬と笛",
        "https://www.aozora.gr.jp/cards/000879/files/56_ruby_845.zip")

similar("太宰 治:純真",
        "https://www.aozora.gr.jp/cards/000035/files/46599_ruby_24668.zip")

similar("夏目 漱石:一夜",
        "https://www.aozora.gr.jp/cards/000148/files/1086_ruby_5742.zip")



--- 「宮沢 賢治:よだかの星」と似た作品は? ---
[('宮澤 賢治:どんぐりと山猫', 0.9996234774589539), ('宮澤 賢治:やまなし', 0.9996147155761719), ('芥川 竜之介:老年', 0.9995146989822388)]

--- 「芥川 龍之介:犬と笛」と似た作品は? ---
[('芥川 竜之介:羅生門', 0.9995617866516113), ('芥川 竜之介:老年', 0.9993518590927124), ('夏目 漱石:吾輩は猫である', 0.9991893768310547)]

--- 「太宰 治:純真」と似た作品は? ---
[('芥川 竜之介:羅生門', 0.9994699954986572), ('芥川 竜之介:老年', 0.9991984367370605), ('宮澤 賢治:どんぐりと山猫', 0.998833417892456)]

--- 「夏目 漱石:一夜」と似た作品は? ---
[('夏目 漱石:虞美人草', 0.9990417957305908), ('夏目 漱石:草枕', 0.9968736171722412), ('太宰 治:お伽草紙', 0.9962616562843323)]



In [21]:
#マルコフ連鎖を使用した自動作文bot

import MeCab
import os,json,random

dict_file = "markov_dict.json"
dic = {}

# 辞書への登録
def regist_dic(wordlist):
    global dic
    w1 = ""
    w2 = ""
    
    # 要素が3未満の場合は、何もしない
    if len(wordlist) < 3 : return
    
    for w in wordlist :
        word = w[0]
        if word == "" or  word == "\r\n" or word == "\n" : continue
        # 辞書に単語を設定
        if w1 and w2 :
            set_dic(dic,w1, w2, word)
        # 文末を表す語のの場合、連鎖をクリアする
        if word == "。" or word == "?" or  word == "？" :
            w1 = ""
            w2 = ""
            continue
        # 次の前後関係を登録するために、単語をスライド
        w1, w2 = w2, word
    
    # 辞書を保存
    json.dump(dic, open(dict_file,"w", encoding="utf-8"))

# 辞書に単語を設定
def set_dic(dic, w1, w2, w3):
    # 新しい単語の場合は、新しい辞書オブジェクトを作成
    if w1 not in dic : dic[w1] = {}
    if w2 not in dic[w1] : dic[w1][w2] = {}
    if w3 not in dic[w1][w2]: dic[w1][w2][w3] = 0
    # 単語の出現数をインクリメントする
    dic[w1][w2][w3] += 1

# 応答文の作成 
def make_response(word):
    res = []
    
    # 「名詞」/「形容詞」/「動詞」は、文章の意図を示していることが多いと想定し、始点の単語とする。
    w1 = word
    res.append(w1)
    w2 = word_choice(dic[w1])
    res.append(w2)
    while True:
        # w1,w2の組み合わせから予想される、単語を選択
        if w1 in dic and w2 in dic[w1] : w3 = word_choice(dic[w1][w2])
        else : w3 = ""
        res.append(w3)
        # 文末を表す語の場合、作文を終了
        if w3 == "。" or w3 == "?" or  w3 == "？"  or w3 == "" :  break
        # 次の単語を選択するために、単語をスライド
        w1, w2 = w2, w3
    return "".join(res)
        
def word_choice(candidate):
    keys = candidate.keys()
    return random.choice(list(keys))

# メイン処理 

# 辞書がすでに存在する場合は、最初に読み込む
if os.path.exists(dict_file):
        dic = json.load(open(dict_file,"r"))
        
while True:
    # 標準入力から入力を受け付け、「さようなら」が入力されるまで続ける
    text = input("You -> ")
    if text == "" or text == "さようなら" : 
        print("Bot -> さようなら")
        break

    # 文章整形
    if text[-1] != "。" and text[-1] != "?" and text[-1] != "？" : text +="。"
    
    # 形態素解析
    tagger = MeCab.Tagger(" /var/lib/mecab/dic/mecab-ipadic-neologd")
    tagger.parse("") 
    node =  tagger.parseToNode(text)
    
    # 形態素解析の結果から、単語と品詞情報を抽出
    wordlist = []
    while node is not None:
        hinshi = node.feature.split(",")[0]
        if  hinshi not  in ["BOS/EOS"]:
            wordlist.append([node.surface,hinshi])
        node = node.next
    
    # マルコフ連鎖の辞書に登録
    regist_dic(wordlist)

    # 応答文の作成
    for w in wordlist:
        word = w[0]
        hinshi = w[1]
        # 品詞が「感動詞」の場合は、単語をそのまま返す
        if hinshi in [ "感動詞"] : 
            print("Bot -> " + word)
            break
        # 品詞が「名詞」「形容詞」「動詞」の場合で、かつ、辞書に単語が存在する場合は、作文して返す
        elif (hinshi in [ "名詞" ,"形容詞","動詞"]) and (word in dic):
            print("Bot -> " + make_response(word))
            break

You -> おはよう
Bot -> おはよう
You -> 明日晴れるかな
Bot -> 明日は雨ですか。
You -> 
Bot -> さようなら


In [6]:
# 全てのテキストを巡回して単語データベースを作成する
import os, glob
import MeCab
import numpy as np
import pickle

# 保存ファイル名
savefile = "./ok-spam.pickle"
# MeCabの準備 
tagger = MeCab.Tagger()
# 変数の準備 
word_dic = {"__id": 0} # 単語辞書
files = [] # 読み込んだ単語データを追加する

# 指定したディレクトリ内のファイル一覧を読む 
def read_files(dir, label):
    # テキストファイルの一覧を得る
    files = glob.glob(dir + '/*.txt')
    for f in files:
        read_file(f, label)

# ファイルを読む 
def read_file(filename, label):
    words = []
    # ファイルの内容を読む
    with open(filename, "rt", encoding="utf-8") as f:
        text = f.read()
    files.append({
        "label": label,
        "words": text_to_ids(text)
    })

# テキストを単語IDのリストに変換
def text_to_ids(text):
    # 形態素解析 
    word_s = tagger.parse(text)
    words = []
    # 単語を辞書に登録 
    for line in word_s.split("\n"):
        if line == 'EOS' or line == '': continue
        word = line.split("\t")[0]
        params = line.split("\t")[1].split(",")
        hinsi = params[0] # 品詞
        hinsi2 = params[1] # 品詞の説明
        org = params[6] # 単語の原型
        # 助詞・助動詞・記号・数字は捨てる 
        if not (hinsi in ['名詞', '動詞', '形容詞']): continue
        if hinsi == '名詞' and hinsi2 == '数': continue
        # 単語をidに変換 
        id = word_to_id(org)
        words.append(id)
    return words

# 単語をidに変換 
def word_to_id(word):
    # 単語が辞書に登録されているか？
    if not (word in word_dic):
        # 登録されていないので新たにIDを割り振る
        id = word_dic["__id"]
        word_dic["__id"] += 1
        word_dic[word] = id
    else:
        # 既存の単語IDを返す
        id = word_dic[word]
    return id

# 単語の頻出頻度のデータを作る 
def make_freq_data_allfiles():
    y = []
    x = []
    for f in files:
        y.append(f['label'])
        x.append(make_freq_data(f['words']))
    return y, x

def make_freq_data(words):
    # 単語の出現回数を調べる
    cnt = 0
    dat = np.zeros(word_dic["__id"], 'float')
    for w in words:
        dat[w] += 1
        cnt += 1
    # 回数を出現頻度に直す
    dat = dat / cnt
    return dat

# ファイルの一覧から学習用のデータベースを作る
if __name__ == "__main__":
    read_files("ok", 0)
    read_files("spam", 1)
    y, x = make_freq_data_allfiles()
    # ファイルにデータを保存
    pickle.dump([y, x, word_dic], open(savefile, 'wb'))
    print("ok")



ok


In [7]:
#学習とテストを繰り返して単語の頻出度を調べる

import pickle
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# データファイルの読込
data_file = "./ok-spam.pickle"
save_file = "./ok-spam-model.pickle"
data = pickle.load(open(data_file, "rb"))
y = data[0] # ラベル
x = data[1] # 単語の出現頻度

# 100回、学習とテストを繰り返す 
count = 100
rate = 0
for i in range(count):
    # データを学習用とテスト用に分割 
    x_train, x_test, y_train, y_test = train_test_split(
        x, y, test_size=0.2)
    # 学習する
    model = GaussianNB()
    model.fit(x_train, y_train)
    # 評価する
    y_pred = model.predict(x_test)
    acc = accuracy_score(y_test, y_pred)
    # 評価結果が良ければモデルを保存 
    if acc > 0.94: pickle.dump(model, open(save_file, "wb"))
    print(acc)
    rate += acc
# 平均値を表示 
print("----")
print("average=", rate / count)



0.9393939393939394
0.9090909090909091
0.9393939393939394
0.9696969696969697
0.9696969696969697
0.9696969696969697
0.8787878787878788
0.9090909090909091
0.9696969696969697
0.9090909090909091
1.0
0.9696969696969697
0.9090909090909091
0.9090909090909091
1.0
0.9393939393939394
0.9393939393939394
0.9090909090909091
0.9696969696969697
0.9393939393939394
0.8787878787878788
0.8181818181818182
0.9090909090909091
0.9696969696969697
0.8787878787878788
0.9393939393939394
0.9696969696969697
0.9696969696969697
0.8181818181818182
1.0
0.9393939393939394
1.0
0.9090909090909091
0.8484848484848485
0.9090909090909091
0.9393939393939394
0.9696969696969697
0.9696969696969697
1.0
0.8484848484848485
1.0
0.9393939393939394
0.9090909090909091
1.0
0.9696969696969697
0.8787878787878788
0.9696969696969697
1.0
0.9696969696969697
1.0
0.9696969696969697
0.9393939393939394
0.8484848484848485
0.9090909090909091
0.9090909090909091
0.9090909090909091
0.8787878787878788
0.9393939393939394
0.9696969696969697
0.939393939393

In [15]:
#スパム判定

import pickle
import MeCab
import numpy as np
from sklearn.naive_bayes import GaussianNB

# テストするテキスト
test_text1 = """
明日晴れるかな
"""
test_text2 = """
億万長者になる方法を教えます。
すぐに以下のアドレスに返信して。
"""
# ファイル名
data_file = "./ok-spam.pickle"
model_file = "./ok-spam-model.pickle"
label_names = ['OK', 'SPAM']
# 単語辞書を読み出す 
data = pickle.load(open(data_file, "rb"))
word_dic = data[2]
# MeCabの準備
tagger = MeCab.Tagger()
# 学習済みモデルを読み出す 
model = pickle.load(open(model_file, "rb"))

# テキストがスパムかどうか判定する 
def check_spam(text):
    # テキストを単語IDのリストに変換し単語の頻出頻度を調べる
    zw = np.zeros(word_dic['__id'])
    count = 0
    s = tagger.parse(text)
    # 単語毎の回数を加算 
    for line in s.split("\n"):
        if line == "EOS": break
        params = line.split("\t")[1].split(",")
        org = params[6] # 単語の原型
        if org in word_dic:
            id = word_dic[org]
            zw[id] += 1
            count += 1
    zw = zw / count
    # 予測 
    pre = model.predict([zw])[0]
    print("- 結果=", label_names[pre])

if __name__ == "__main__": 
    check_spam(test_text1)
    check_spam(test_text2)


- 結果= OK
- 結果= SPAM
